## Homework: Testing for Positive Beta Premium in REITs

In this homework, you will test for a positive beta premium in REIT markets, which means that you will test that the market as a whole provides a higher return than the risk free rate.  You will use an equal-weighted REIT portfolio to represent the REIT market (note that in tutorial 2 we used the market-cap weighted portfolio), and you will use the one-month treasury rate to represent the risk-free rate.  Here are the data sets that you will use:

1. `DGS1MO.csv`: one-month treasury bills.
2. `universe.csv`: set of names that you will use to construct the REIT market.
3. `reit_px.csv`: daily prices for all REITs in the universe. 

Between the two data sources, you have enough data to test for the period 8/2001 - 12/2019.  As a hint, my calculations found that over this time period, the REIT market returned 13.29% per annum, while treasuries returned 1.32% per annum.

The deliverable you turn in will be a Jupyter Note book in a zipped folder (the same way I deliver tutorials).  The folder should contain all data involved in the analysis.  The notebook should run from start to finish without any modification to the code.  If your notebook does not run correctly, you will be docked points in an amount proportional to the amount of inconvenience it takes to get it to run, or you may be required to resubmit your homework entirely, and it will be counted late, and will be docked points. I leave both these decisions entirely to the TAs disgression.

## Load Packages

In [1]:
import pandas as pd
import numpy as np

## Treasury Data

In [2]:
df_rf = pd.read_csv('DGS1MO.csv')
df_rf.head()

,DATE,DGS1MO
0,2001-07-31,3.67
1,2001-08-01,3.65
2,2001-08-02,3.65
3,2001-08-03,3.63
4,2001-08-06,3.62


As a matter of style, I prefer column names to be all lowercase.

In [26]:
df_rf.columns = df_rf.columns.str.lower()

There are a couple of bad data points that I simply remove, this doesn't seem to cause issues with the analysis further down the line.

In [27]:
df_rf = df_rf[df_rf['dgs1mo'] != "."]

Next, let's change data types for a couple of columns.

In [28]:
df_rf['dgs1mo'] = df_rf['dgs1mo'].astype(float)
df_rf['date'] = pd.to_datetime(df_rf['date'])

Let's extract the year and month from the `date` column for groupbing later.

In [8]:
df_rf['year'] = df_rf['date'].dt.year
df_rf['month'] = df_rf['date'].dt.month

For a given month, we will use the rate from the last day of the previous month.

In [29]:
df_bom_rf = df_rf.groupby(['year', 'month'])[['date']].max().reset_index()
df_bom_rf['as_of_date'] = df_bom_rf['date'].shift()
df_bom_rf.head()

,year,month,date,as_of_date
0,2001,7,2001-07-31,NaT
1,2001,8,2001-08-31,2001-07-31
2,2001,9,2001-09-28,2001-08-31
3,2001,10,2001-10-31,2001-09-28
4,2001,11,2001-11-30,2001-10-31


Let's remove the first month, which we don't have data for.

In [12]:
df_bom_rf = df_bom_rf.query('date != "2001-07-31"')
df_bom_rf.head()

,year,month,date,as_of_date
1,2001,8,2001-08-31,2001-07-31
2,2001,9,2001-09-28,2001-08-31
3,2001,10,2001-10-31,2001-09-28
4,2001,11,2001-11-30,2001-10-31
5,2001,12,2001-12-31,2001-11-30


Using a `.merge()` to grab the rate from the `as_of_date` of each month.

In [30]:
df_1m_treasury = \
    (
    df_bom_rf[['year', 'month', 'as_of_date']]
        .merge(df_rf[['date', 'dgs1mo']], left_on='as_of_date', right_on = 'date')
        .assign(month_ret = lambda df: df.dgs1mo / 100 / 12) # rates are given in annual terms, so we divide by 12
        .assign(asset = 'risk_free')
        .query('year < 2020') # our REIT price data does not include 2020
        [['asset', 'year', 'month', 'month_ret']]
    )
df_1m_treasury.head()

,asset,year,month,month_ret
0,risk_free,2001,8,0.003058
1,risk_free,2001,9,0.002833
2,risk_free,2001,10,0.001900
3,risk_free,2001,11,0.001792
4,risk_free,2001,12,0.001558


## REIT Data

We begin by reading-in the universe.

In [14]:
df_universe = pd.read_csv('universe.csv')
df_universe = df_universe[['symbol', 'reit_name', 'type', 'sector']]
df_universe.head()

,symbol,reit_name,type,sector
0,ANH,Anworth Mortgage Asset Corporation,Mortgage REIT,Mortgage
1,BXMT,"Blackstone Mortgage Trust, Inc.",Mortgage REIT,Mortgage
2,CMO,Capstead Mortgage Corporation,Mortgage REIT,Mortgage
3,DX,"Dynex Capital, Inc.",Mortgage REIT,Mortgage
4,MFA,"MFA Mortgage Investments, Inc.",Mortgage REIT,Mortgage


Next, let's read-in the price data.

In [15]:
df_px_all = pd.read_csv('reit_px.csv')
df_px_all['date'] = pd.to_datetime(df_px_all['date'])
df_px_all.head()

,date,open,high,low,close,volume,adjusted,ticker
0,2004-04-07,20.049999,20.600000,20.049999,20.50,3707500.0,6.464799,ABR
1,2004-04-08,20.500000,20.500000,20.360001,20.43,301200.0,6.442722,ABR
2,2004-04-12,20.450001,20.450001,19.420000,19.52,852900.0,6.155750,ABR
3,2004-04-13,19.510000,19.549999,19.350000,19.51,226800.0,6.152596,ABR
4,2004-04-14,19.400000,19.480000,19.400000,19.43,143900.0,6.127368,ABR


There is more data in `df_px_all` than we will need, so let's filter this down using various means.

In [16]:
df_px = \
    (
    df_px_all
        .merge(df_universe[['symbol']], right_on = 'symbol', left_on = 'ticker') # limiting to tickers in our universe
        .query('date >= "1998-12-31"') # limiting date range
        .query('date < "2020-01-01"')  # limiting date range
        .assign(year = lambda df: df.date.dt.year) # pulling out year for grouping later
        .assign(month = lambda df: df.date.dt.month) # pulling out month for grouping later
        [['symbol', 'year', 'month', 'date', 'close', 'adjusted']] # keeping only the columns we need
        .sort_values(['symbol', 'year', 'month']) # sorting to make sure our returns calculations are correct
    )
df_px.head()

,symbol,year,month,date,close,adjusted
74050,ADC,1998,12,1998-12-31,18.5000,4.091326
74051,ADC,1999,1,1999-01-04,18.2500,4.036036
74052,ADC,1999,1,1999-01-05,17.7500,3.925461
74053,ADC,1999,1,1999-01-06,17.6875,3.911640
74054,ADC,1999,1,1999-01-07,17.7500,3.925461


It will be easiest for us to calculate monthly returns from daily returns. Therefore, we will next calculate daily returns for each `symbol` using a `groupby()`.

In [17]:
df_px['dly_ret'] = df_px.groupby(['symbol'])['adjusted'].pct_change()
df_px.head()

,symbol,year,month,date,close,adjusted,dly_ret
74050,ADC,1998,12,1998-12-31,18.5000,4.091326,NaN
74051,ADC,1999,1,1999-01-04,18.2500,4.036036,-0.013514
74052,ADC,1999,1,1999-01-05,17.7500,3.925461,-0.027397
74053,ADC,1999,1,1999-01-06,17.6875,3.911640,-0.003521
74054,ADC,1999,1,1999-01-07,17.7500,3.925461,0.003533


In order to calculate multi-day returns from single day returns, we will need the following user-defined function.

In [18]:
def cum_ret(ret):
    return(np.prod(1 + ret) - 1)

With the `cum_ret()` function created, let's now calculate the monthly returns from the daily returns by again using the `groupby()` function.

In [19]:
df_month_ret = \
    (
    df_px
        .query('year > 1998')
        .groupby(['year', 'month', 'symbol'])[['dly_ret']].agg(cum_ret)
        .reset_index()
        .rename(columns={'dly_ret':'month_ret'})
    )
df_month_ret.head()

,year,month,symbol,month_ret
0,1999,1,ADC,-0.054054
1,1999,1,AI,-0.019231
2,1999,1,AIV,-0.001681
3,1999,1,AKR,-0.023810
4,1999,1,ALX,-0.078337


We can now calculate monthly return of the equal-weighted portfolio, which is simple the `.mean()` of the monthly returns of the individual REITs.

In [20]:
df_reit_mkt_eqw = \
    (
    df_month_ret
        .groupby(['year', 'month']).mean().reset_index()
        .assign(asset = 'reit_mkt_eqw')
        .assign(year_month = lambda df: (df.year * 100) + df.month)
        .query('year_month >= 200109')
        [['asset', 'year', 'month', 'month_ret']]
    )
df_reit_mkt_eqw.head()

,asset,year,month,month_ret
32,reit_mkt_eqw,2001,9,-0.013307
33,reit_mkt_eqw,2001,10,0.003374
34,reit_mkt_eqw,2001,11,0.031865
35,reit_mkt_eqw,2001,12,0.042843
36,reit_mkt_eqw,2002,1,0.024740


## Annualized Return

We can now calculate the annualized return of our two assets. As you can see, over this time period, the REIT market to return 13.29% per annum, while treasuries returned 1.32% per annum.

In [24]:
(
pd.concat([df_1m_treasury, df_reit_mkt_eqw])
    .groupby(['asset'])[['month_ret']].agg(cum_ret).reset_index().rename(columns={'month_ret':'cum_ret'})
    .assign(ann_ret = lambda df: (df.cum_ret + 1) ** (12 / 220) - 1)
)

,asset,cum_ret,ann_ret
0,reit_mkt_eqw,8.852098,0.132902
1,risk_free,0.270678,0.013152
